In [1]:
from logicqubit.logic import *
from cmath import *
import numpy as np
import sympy as sp
from scipy.optimize import *
import matplotlib.pyplot as plt

logicqubit-gpu version 1.4.4


In [2]:
def qft(qr):
    for i in range(len(qr)):
        for j in range(i):
            qr[i].CU1(qr[j], pi/float(2**(i-j)))
        qr[i].H()
        
def iqft(qr): # transformada quântica de Fourier inversa
    for i in range(len(qr)):
        for j in range(i):
            qr[i].CU1(qr[j], -pi/float(2**(i-j)))
        qr[i].H()
        
def swap(s1, s2):
    s2.CX(s1)
    s1.CX(s2)
    s2.CX(s1)
    
#def ansatz(q, params):
#    return q.RY(params[0])

def _ansatz(reg, params):
    n_qubits = len(reg)
    depth = n_qubits
    for i in range(depth):
        reg[1].CNOT(reg[0])
        for j in range(n_qubits):
            reg[i].RY(params[j])
            
def ansatz(reg, params):
    n_qubits = len(reg)
    depth = n_qubits
    for i in range(depth):
        for j in range(n_qubits):
            if(j < n_qubits-1):
                reg[j+1].CNOT(reg[j])
            reg[i].RY(params[j])
            
def ansatz_2(q1, q2, params):
    q2.CNOT(q1)
    q1.RY(params[0])
    q2.RY(params[1])
    q1.CNOT(q2)
    q1.RY(params[0])
    q2.RY(params[1])
    q2.CNOT(q1)
    q1.RY(params[0])
    q2.RY(params[1])

In [3]:
def expectation(params):
    # H = [[1,0,0,0],[0,-1,0,0],[0,0,-5,0],[0,0,0,1]]
    # <psi|H|psi> = <psi|00><00|psi> - <psi|01><01|psi> - 5*<psi|10><10|psi> + <psi|11><11|psi>
    logicQuBit  = LogicQuBit(2, first_left=True)
    #reg = QubitRegister(2)
    q1 = Qubit()
    q2 = Qubit()
       
    #_ansatz(reg.getQubits(), params)
    #ansatz([q2,q1], params)
    ansatz_2(q1,q2,params)
    
    res = logicQuBit.Measure([q1,q2])
                      
    return -5*res[0]+res[1]-4*res[2]-10*res[3]

In [4]:
expectation([0,0])

-5.0

In [5]:
minimum = minimize(expectation, [0,0], method='Nelder-Mead')
print(minimum)

 final_simplex: (array([[1.21267393, 0.32226075],
       [1.21263944, 0.32234993],
       [1.21262322, 0.32233524]]), array([-7.5783574, -7.5783574, -7.5783574]))
           fun: -7.578357400998107
       message: 'Optimization terminated successfully.'
          nfev: 124
           nit: 66
        status: 0
       success: True
             x: array([1.21267393, 0.32226075])
